# Visualization notebook



## Imports and functions

In [84]:
import utils
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

# 1,071,477 authors

# Exploring MUD

In [86]:
import ijson
import sqlite3

data = ijson.parse(open("data/mud/full/raw_mud/raw_all/data.jsonl"),multiple_values=True)

x = 0
d = {}
posts = []
for prefix, event, value in data:
    
    if prefix == "syms.item":
        posts.append(value)
        
    if prefix.startswith("author_id"):
        d[value] = posts
        posts = []
        x += 1

    if x == 1:
        break


utils.save_json(d, "example_author.json")

4965